### Parallelization with Dask

In [1]:
import numpy as np
import ipyparallel as ipp
import itertools
from distributed import progress
import pandas as pd
from typing import NamedTuple

import smpsite.smpsite as smp

In [2]:
rc = ipp.Cluster(n=40).start_and_connect_sync()

Starting 40 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/40 [00:00<?, ?engine/s]

In [3]:
dask_client = rc.become_dask()
dask_client

<Client: 'tcp://192.168.41.232:45311' processes=0 threads=0, memory=0 B>

In [4]:
# Check to see if the threads are ready
dview = rc[:]
len(dview)

40

In [5]:
def ipp_simulate_estimations(kappa_within_site, 
                             site_lat,
                             outlier_rate,
                             n,
                             k,
                             site_lon,
                             ignore_outliers,
                             seed):
    
    # If we don't include the definition of the class inside the code, this returns this weird message error... why? 
    class Params(NamedTuple):

        kappa_within_site : float    
        site_lat : float
        outlier_rate : float
        n : int
        k : int    
        site_lon : float

    params = Params(kappa_within_site=kappa_within_site, 
                    site_lat=site_lat,
                    outlier_rate=outlier_rate,
                    n=n,
                    k=k,
                    site_lon=site_lon)
    
    df_tot = smp.simulate_estimations(params, 
                                      n_iters=1000, 
                                      ignore_outliers=ignore_outliers,
                                      seed=seed)
    
    stats = dict(df_tot['error_angle'].describe())
    df = pd.DataFrame.from_dict({'error_angle_mean': [stats['mean']], 
                                 'error_angle_median': [stats['50%']], 
                                 'error_angle_std': [stats['std']]})
    
    df['N'] = n * k
    df['k'] = k
    df['n'] = n
    df['kappa_within_site'] = kappa_within_site
    df['site_lat'] = site_lat
    df['outlier_rate'] = outlier_rate
    df['ignore_outliers'] = ignore_outliers
    df["site_long"] = site_lon
    
    return df

We need to set the structure for the parallel simulations with different parameters. 

In [6]:
min_N = 10
max_N = 100

all_angular_within_site = [10]
all_kappa_within_site = [smp.angular2kappa(deg) for deg in all_angular_within_site]
all_site_lat = [30]#np.arange(0, 91, 10)
all_outlier_rate = [0.00, 0.10]
all_n = np.arange(1,21)
all_k = np.arange(1,21)
all_site_lon = [0]
all_ignore_outliers = [False, True]

all_kappa_within_site, all_site_lat, all_outlier_rate, all_n, \
all_k, all_site_lon, all_ignore_outliers \
= np.meshgrid(all_kappa_within_site, all_site_lat, \
                       all_outlier_rate, all_n, all_k, \
                         all_site_lon, all_ignore_outliers)

all_kappa_within_site = all_kappa_within_site.ravel()
all_site_lat = all_site_lat.ravel()
all_outlier_rate = all_outlier_rate.ravel()
all_n = all_n.ravel()
all_k = all_k.ravel()
all_site_lon = all_site_lon.ravel()
all_ignore_outliers = all_ignore_outliers.ravel()

all_N = all_n * all_k 
valid_index = (min_N <= all_N) & (all_N < max_N) 

all_kappa_within_site = all_kappa_within_site[valid_index]
all_site_lat = all_site_lat[valid_index]
all_outlier_rate = all_outlier_rate[valid_index]
all_n = all_n[valid_index]
all_k = all_k[valid_index]
all_site_lon = all_site_lon[valid_index]
all_ignore_outliers = all_ignore_outliers[valid_index]


n_tasks = len(all_n)
all_seed = np.random.randint(0, 2**32-1, n_tasks)

In [7]:
len(all_kappa_within_site), len(all_site_lat), len(all_outlier_rate), \
len(all_n), len(all_k), len(all_site_lon), len(all_ignore_outliers)

(800, 800, 800, 800, 800, 800, 800)

In [8]:
task = dask_client.map(ipp_simulate_estimations, all_kappa_within_site, \
                       all_site_lat, all_outlier_rate, all_n, \
                       all_k, all_site_lon, all_ignore_outliers, all_seed)

res = dask_client.submit(pd.concat, task)

progress(res)

VBox()

In [9]:
df_all = res.result()
df_all.to_csv('outputs/run3.csv')

In [1]:
df_all

NameError: name 'df_all' is not defined